In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Lagrangian Neural Network for Block Sliding on a Slope
class LagrangianNN(nn.Module):
    def __init__(self):
        super(LagrangianNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, q, q_dot):
        inputs = torch.cat([q, q_dot], dim=1)
        return self.fc(inputs)

# Loss function for Block Sliding on a Slope
def lagrangian_loss(model, q, q_dot, q_ddot, g=9.81, m=1.0, alpha=0.5):
    L = model(q, q_dot)
    dL_dq_dot = torch.autograd.grad(L.sum(), q_dot, create_graph=True)[0]
    dL_dq = torch.autograd.grad(L.sum(), q, create_graph=True)[0]
    d_dL_dq_dot_dt = torch.autograd.grad(dL_dq_dot.sum(), q_dot, create_graph=True)[0]
    predicted_q_ddot = -dL_dq + d_dL_dq_dot_dt

    true_q_ddot = -g * torch.sin(alpha) * torch.ones_like(q)
    return ((predicted_q_ddot - true_q_ddot) ** 2).mean()

# Training Block Sliding on a Slope
model = LagrangianNN()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

q_data = torch.linspace(0, 1, 100).view(-1, 1)
q_dot_data = torch.linspace(0, 1, 100).view(-1, 1)
q_ddot_data = -9.81 * torch.sin(0.5) * torch.ones_like(q_data)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = lagrangian_loss(model, q_data, q_dot_data, q_ddot_data)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Define the Lagrangian Neural Network for Double Pendulum
class DoublePendulumLNN(nn.Module):
    def __init__(self):
        super(DoublePendulumLNN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

    def forward(self, theta1, theta2, theta1_dot, theta2_dot):
        inputs = torch.cat([theta1, theta2, theta1_dot, theta2_dot], dim=1)
        return self.fc(inputs)

# Loss function for Double Pendulum
def double_pendulum_loss(model, theta1, theta2, theta1_dot, theta2_dot, g=9.81, m1=1.0, m2=1.0, l1=1.0, l2=1.0):
    L = model(theta1, theta2, theta1_dot, theta2_dot)
    dL_dtheta1_dot = torch.autograd.grad(L.sum(), theta1_dot, create_graph=True)[0]
    dL_dtheta2_dot = torch.autograd.grad(L.sum(), theta2_dot, create_graph=True)[0]
    dL_dtheta1 = torch.autograd.grad(L.sum(), theta1, create_graph=True)[0]
    dL_dtheta2 = torch.autograd.grad(L.sum(), theta2, create_graph=True)[0]

    d_dL_dtheta1_dot_dt = torch.autograd.grad(dL_dtheta1_dot.sum(), theta1_dot, create_graph=True)[0]
    d_dL_dtheta2_dot_dt = torch.autograd.grad(dL_dtheta2_dot.sum(), theta2_dot, create_graph=True)[0]

    predicted_theta1_ddot = -dL_dtheta1 + d_dL_dtheta1_dot_dt
    predicted_theta2_ddot = -dL_dtheta2 + d_dL_dtheta2_dot_dt

    true_theta1_ddot = torch.zeros_like(theta1)
    true_theta2_ddot = torch.zeros_like(theta2)

    return ((predicted_theta1_ddot - true_theta1_ddot) ** 2).mean() + \
           ((predicted_theta2_ddot - true_theta2_ddot) ** 2).mean()

# Training Double Pendulum LNN
model_dp = DoublePendulumLNN()
optimizer_dp = optim.Adam(model_dp.parameters(), lr=1e-3)

theta1_data = torch.linspace(0, 2 * torch.pi, 100).view(-1, 1)
theta2_data = torch.linspace(0, 2 * torch.pi, 100).view(-1, 1)
theta1_dot_data = torch.zeros_like(theta1_data)
theta2_dot_data = torch.zeros_like(theta2_data)

for epoch in range(1000):
    optimizer_dp.zero_grad()
    loss_dp = double_pendulum_loss(model_dp, theta1_data, theta2_data, theta1_dot_data, theta2_dot_data)
    loss_dp.backward()
    optimizer_dp.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss_dp.item():.4f}")
